In [ ]:

# import

# basic 
import numpy as np
import scipy
import pandas as pd
import math

import matplotlib.pyplot as plt
plt.style.use('seaborn')

# there also is a module on preprocessing in sklearn

##sklearn learners (learners as the different types of models)
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

##sklearn metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

##sklearn model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV

In [ ]:
# read in boston data
bd = pd.read_csv("https://bitbucket.org/remcc/rob-data-sets/downloads/Boston.csv")

#pull off y=medv and x = lstat
y = bd['medv']
X = bd['lstat'].to_numpy()[:,np.newaxis]

#plot x vs. y
plt.scatter(X,y)
plt.xlabel('lstat')
plt.ylabel('mdev')

In [ ]:
# fit one knn

# create model object setting the hyperparameter n_neighbors to 50
knnmod = KNeighborsRegressor(n_neighbors=50)

# fit with training data
knnmod.fit(X,y)

#predict on sorted x training values
Xtest = np.sort(X[:,0])[:,np.newaxis]
yhat = knnmod.predict(Xtest)

In [ ]:
#plot fit
plt.scatter(X,y,s=10,c='k',marker='.')
plt.plot(Xtest,yhat,c='red')
plt.xlabel('lstat')
plt.ylabel('medv')

In [ ]:
#  cross validation using sklearn cross_val_score

# to see a list of scorers:
# sorted(sklearn.metrics.SCORERS.keys()) 

#model object
tempmod = KNeighborsRegressor(n_neighbors=40) #knn with k=40

## rmse from cross validation
cvres = cross_val_score(tempmod,X,y,cv=5,scoring='neg_mean_squared_error') #cross val with 5 folds

In [ ]:
# train/test split

#train/test split
myseed = 34
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,random_state=myseed, test_size=.2)

# model object
kmod = KNeighborsRegressor(n_neighbors=50)

# fit on train
kmod.fit(Xtrain,ytrain)

# predict on test
ypred = kmod.predict(Xtest)

In [ ]:
# tranform to rmse
rmse = math.sqrt(np.mean(-cvres)) 
print('the rmse for k=40 based on 5-fold is:', rmse)

In [ ]:
#  loop over k using simple train/test split

Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,random_state=myseed, test_size=.2)

kvec = np.arange(348) + 2 #values of k to try
ormsev = np.zeros(len(kvec)) # storage for oos rsmse
irmsev = np.zeros(len(kvec)) # storage for in-sample rsmse

for i in range(len(kvec)):
   tmod = KNeighborsRegressor(n_neighbors=kvec[i])
   tmod.fit(Xtrain,ytrain)
   yhat = tmod.predict(Xtest)
   ormsev[i] = math.sqrt(mean_squared_error(ytest,yhat))
   yhat = tmod.predict(Xtrain)
   irmsev[i] = math.sqrt(mean_squared_error(ytrain,yhat))

In [ ]:
# plot rmse vs k
plt.scatter(kvec,ormsev,c='blue',label='out-of-sample')
plt.plot(kvec,irmsev,c='red',label='in-sample')
plt.xlabel('k'); plt.ylabel('rmse')
plt.legend()

In [ ]:
# plot rmse vs model complexity
mcmp = np.log(1/kvec) #model complexity
plt.scatter(mcmp,ormsev,c='blue',label='out-of-sample')
plt.plot(mcmp,irmsev,c='red',label='in-sample')
plt.xlabel('model complexity = log(1/k)',size='x-large')
plt.ylabel('rmse',size='x-large')
plt.title('rmse: blue: out of sample, red: in sample',size='x-large')
plt.legend()

In [ ]:

#  cross validation using sklearn cross_val_score

# to see a list of scorers:
# sorted(sklearn.metrics.SCORERS.keys()) 

#model object
tempmod = KNeighborsRegressor(n_neighbors=40) #knn with k=40

## rmse from cross validation
cvres = cross_val_score(tempmod,X,y,cv=5,scoring='neg_mean_squared_error') #cross val with 5 folds

# tranform to rmse
rmse = math.sqrt(np.mean(-cvres)) 
print('the rmse for k=40 based on 5-fold is:', rmse)

## do it again but shuffle the data
np.random.seed(34) 
indices = np.random.choice(X.shape[0],X.shape[0],replace=False)
ys = y[indices]
Xs = X[indices,:]
cvres = cross_val_score(tempmod,Xs,ys,cv=5,scoring='neg_mean_squared_error')
rmse = math.sqrt(np.mean(-cvres))
print('the rmse for k=40 based on 5-fold is:', rmse)

In [ ]:

#  cross validation on a grid of k values using sklearn validation_curve function

# create the knn model
model = KNeighborsRegressor() # create the knn model

# do cv at every value of k in kvec
# each row of (trains,test)S will correspond to a value of k
# each column has the cv=10 neg_mean_squared_error in-sample (trainS) and out of sample (testS)
trainS, testS = validation_curve(model,X,y,'n_neighbors',kvec,cv=10,scoring='neg_mean_squared_error')

# transform neg_mean_squared_error to rmse
trrmse = np.sqrt(-trainS.mean(axis=1))
termse = np.sqrt(-testS.mean(axis=1))

In [ ]:
#plot in and out of sample rmse
plt.scatter(mcmp,termse,label='in-sample')
plt.plot(mcmp,trrmse,c='red',label='out-of-sample')
plt.xlabel('model complexity = log(1/k)',size='x-large')
plt.ylabel('rmse',size='x-large')
plt.legend()

In [ ]:
#plot to check predictions
plt.scatter(ytest,ypred)
plt.plot(ypred,ypred,c='red')

In [ ]:
#rmse
k50mse = mean_squared_error(ytest,ypred)

#check rmse
check  = np.sum((ypred-ytest)**2)/len(ytest)
print('val from fun:',k50mse,' and check val: ',check)

In [ ]:
# cross val on a grid using sklearn GridSearchCV

# hyperparamter values to try in the gid search, can expand dict to more than one tuning parameter
param_grid={'n_neighbors' : kvec} # searching over n_nerighbors and trying its value kvec

# grid  is the grid searh object
grid = GridSearchCV(model,param_grid,cv=10,scoring='neg_mean_squared_error')

# now run the grid search
grid.fit(X,y)

grid.best_params_ #best value from grid
grid.best_index_ # index of best value from grid

#check
print(kvec[grid.best_index_])

temp = grid.cv_results_ # results from the grid search (a dictionary)
print(temp.keys()) # what is in temp
temp['mean_test_score'] # this is the average score over folds at the values in param_grid

#transform to rmse (notice the negative sign in order to get positive value)
rmsevals = np.sqrt(-temp['mean_test_score'])

In [ ]:
# plot
plt.plot(mcmp,rmsevals) # plot model complexity vs. rmse
plt.xlabel('model complexity = log(1/k)',size='x-large')
plt.ylabel('rmse',size='x-large')
plt.title('rmse from GridSearch')